### Checklist for submission

It is extremely important to make sure that:

1. Everything runs as expected (no bugs when running cells);
2. The output from each cell corresponds to its code (don't change any cell's contents without rerunning it afterwards);
3. All outputs are present (don't delete any of the outputs);
4. Fill in all the places that say `YOUR CODE HERE`, or "**Your answer:** (fill in here)".
5. You **ONLY** change the parts of the code we asked you to, nowhere else (change only the coding parts saying `# YOUR CODE HERE`, nothing else);
6. Don't add any new cells to this notebook;
7. Fill in your group number and the full names of the members in the cell below;
8. Make sure that you are not running an old version of IPython (we provide you with a cell that checks this, make sure you can run it without errors).

Failing to meet any of these requirements might lead to either a subtraction of POEs (at best) or a request for resubmission (at worst).

We advise you the following steps before submission for ensuring that requirements 1, 2, and 3 are always met: **Restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). This might require a bit of time, so plan ahead for this (and possibly use Google Cloud's GPU in HA1 and HA2 for this step). Finally press the "Save and Checkout" button before handing in, to make sure that all your changes are saved to this .ipynb file.

---

Group number and member names:

In [16]:
GROUP = "43"
NAME1 = "Edvin Agnas"
NAME2 = "Hai Dinh"

Make sure you can run the following cell without errors.

In [17]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

---

# HA2 - Recurrent Neural Networks

Welcome to the second group home assignment!  
The purpose of this assigment is to give you practical knowledge in how to implement recurrence in a neural network.  

Every day you are exposed to sequences of data. For example text, video streams, audio, financial time series and medical sensors. Recurrence is therefore an important topic in the field of Machine Learning because it has the potential to solve real-world problems including the types of data above.  

In this assignment, you will learn to:
 - Preprocess text data sets
 - Implement a vanilla RNN cell using only numpy
 - Text generation
 - Build a recurrent neural network with LSTM using Keras  
 - Neural machine translation

**NOTE:** Task 1 (Shallow vanilla RNN) and Task 2 (Neural machine translation), are independent from each other. Task 2 asks you to train a NMT, which takes a while (specially without a GPU), so it might be efficient to start with task 2 and leave it running in the background while you solve task 1.

Table of Contents:  
 [1 Shallow vanilla RNN](#1)  
   [1.1 Preprocessing](#1.1)  
     [1.1.1 Loading dataset](#1.1.1)  
     [1.1.2 One-hot representations](#1.1.2)  
   [1.2 RNN cell class](#1.2)  
   [1.3 Training the RNN](#1.3)  
 [2 Neural machine translation](#2)  
   [2.1 Pre-processing](#2.1)  
     [2.1.1 Loading and inspecting dataset](#2.1.1)  
     [2.1.2 Cleaning the dataset](#2.1.2)  
     [2.1.3 Restricting sentence length and shuffling the data set](#2.1.3)  
     [2.1.4 Word-to-index and index-to-word conversions](#2.1.4)  
     [2.1.5 Padding](#2.1.5)  
     [2.1.6 One-hot labels](#2.1.6)  
   [2.2 Implementing a sequence-to-sequence model](#2.2)  
     [2.2.1 Defining the architecture](#2.2.1)  
     [2.2.2 Training the model](#2.2.2)  
     [2.2.3 Evalutation](#2.2.3)  
     [2.2.4 Testing](#2.2.4)  
     
**NOTE**: The tests available are not exhaustive, meaning that if you pass a test you have avoided the most common mistakes, but it is still not guaranteed that you solution is 100% correct.

## 1 Shallow vanilla RNN <a class="anchor" id="1"></a>
In the first part of this assignment, you will implement a recurrent neural network from scratch without using any framework. You will train this network to predict the next character of a text, which will result in a network that can generate new sentences.  

Start by importing dependencies below  

In [18]:
import numpy as np
import copy
import matplotlib.pyplot as plt
from utils.tests.ha2Tests import *

### 1.1 Preprocessing <a class="anchor" id="1.1"></a>
#### 1.1.1 Loading data set <a class="anchor" id="1.1.1"></a>
The text corpus to train your RNN on is going to be the book The metamorphosis by Franz Kafka. Run the cell below to load the text.

In [19]:
data = open('./utils/kafka.txt', 'r', encoding="utf-8").read()

chars = list(set(data)) 
data_size, vocab_size = len(data), len(chars)
print('data has %d chars, %d unique' % (data_size, vocab_size))

data has 118560 chars, 62 unique


#### 1.1.2 One-hot representations <a class="anchor" id="1.1.2"></a>
`data` is now a string, containing the contents of the book, with 80 unique characters. As usual, converting the labels (that is also the data in our case) into one-hot vectors is a good way for creating unbiased labels.  

Below is a variable `one_hot_vectors` defined that maps every character in `chars` to a unique one-hot vector  

In [21]:
one_hot_vectors = np.eye(len(chars))
char_to_onehot = { val: one_hot_vectors[key,:].reshape(1,-1) for key, val in enumerate(chars) }

### 1.2 RNN cell class <a class="anchor" id="1.2"></a>

Now you will implement the RNN class. For your convenience, this task is split into 4 subtasks, one for each method of the class. These methods will be later linked to a class definition (if you're curious, scroll down to 1.2.5).

Instead of implementing a neural network of arbitrary length like in **IHA1**, you are only going to implement a single RNN cell. At each time-step, the RNN cell will have one input, $\mathbf{x}_t$ , one hidden layer with a hidden state $\mathbf{h}_t$, and one output $\mathbf{y}_t$:

![Simple RNN cell](utils/images/RNN-cell.png)

The output at time-step $t$, referred to as $\mathbf{y}_t$, is computed according to the following equations. Equation 1 and 2 together define the next hidden state $\mathbf{h}_t$, and Equation 3 defines the unnormalized probability output $\mathbf{z}_t$.

$$
\mathbf{z}_t =  \mathbf{x}_t \mathbf{W}_{xh} + \mathbf{h}_{t-1} \mathbf{W}_{hh} + \mathbf{b}_h, \tag{1}
$$
$$
\mathbf{h}_t = \tanh (\mathbf{z}_t), \tag{2}
$$
$$
\mathbf{y}_t =  \mathbf{h}_t \mathbf{W}_{hy} + \mathbf{b}_y, \tag{3}
$$

where $\mathbf{W}_{xh}$, $\mathbf{W}_{hh}$, $\mathbf{W}_{hy}$, $\mathbf{b}_{h}$, and $\mathbf{b}_{y}$ are the parameters of the RNN cell.

Calculating $\mathbf{h}_{30}$ means that you need to need to have calculated every earlier hidden state $\{\mathbf{h}_t | t \in \{0, 1, ..., 29\}\}$. This can easier be understood by always imagining an unrolled RNN cell:
![Simple RNN cell unrolled](utils/images/RNN-cell-unrolled.png)
  

Since it is a multiclass classification problem, the softmax activation function is going to be used to normalize the output of the RNN cell $\mathbf{y}_t$, defined as

$$
\mathbf{p}_t = softmax(\mathbf{y}_t) = \frac{e^{\mathbf{y}_t - max(\mathbf{y_t})}}{ \sum^j e^{y_{j_t} - max(\mathbf{y_t})}}. \tag{4}
$$

#### 1.2.1 The initalization method

The first step in creating the RNN class is to create its `__init__` method, where we create all the necessary attributes and initialize them. Complete the `init_routine` function below. Later this will be linked to the `__init__` method in the `SimpleRNN` class, so you're now developing the constructor of that class.

In [20]:
def init_routine(self, input_dim, hidden_dim, output_dim):
    """
    Initialize the weights of the RNN and define a cache with the starting value of the hidden state

    Arguments:
    self - an object of the class SimpleRNN
    input_dim - an integer representing the number of inputs
    hidden_dim - an integer representing the length of the hidden state vector
    output_dim - an integer representing the number of outputs

    Attributes:
    input_dim - attribute initialized with the value of the argument `input_dim`
    cache - a dictionary holding the values of `h_start`, `xs` and `hs` from the previous forward propagation call.
            h_start - the value to initialize the hidden state with when `forward_prop is called. The value of
                      `h_start` should be initialized to a `numpy.ndarray` vector of zeros 
            xs - the list of `xs` used in the last `forward_prop` call. Later used in `backward_prop`. Can be
                 initialized to `None`
            hs - the list of `hs` calculated in the last `forward_prop` call. Later used in `backward_prop`. Can
                 be initialized to `None`
    W_hh - weight matrix of shape (hidden_dim, hidden_dim) and type `numpy.ndarray`. 
           Initialized randomly from a normal distribution of mean zero and stddev 0.01
    W_xh - weight matrix of shape (input_dim, hidden_dim) and type `numpy.ndarray`. 
           Initialized randomly from a normal distribution of mean zero and stddev 0.01
    W_hy - weight matrix of shape (hidden_dim, output_dim) and type `numpy.ndarray`. 
           Initialized randomly from a normal distribution of mean zero and stddev 0.01
    b_h -  bias of shape (1, hidden_dim) and type `numpy.ndarray`. Initialized to all zeros
    b_y -  bias of shape (1, output_dim) and type `numpy.ndarray`. Initialized to all zeros
    """

    mu, stdev = 0, 0.01
    self.input_dim = input_dim
    self.cache = {
        'h_start': np.zeros((1, hidden_dim)),
        'xs': None,
        'hs': None
    }
    self.W_hh = np.random.normal(mu, stdev, (hidden_dim, hidden_dim))
    self.W_xh = np.random.normal(mu, stdev, (input_dim, hidden_dim))
    self.W_hy = np.random.normal(mu, stdev, (hidden_dim, output_dim))
    self.b_h = np.zeros((1, hidden_dim))
    self.b_y = np.zeros((1, output_dim))
    

The following cell tests if your implementation is correct.

In [22]:
test_init_routine(init_routine)

test init passed


#### 1.2.2 Forward propagation

Now that we have an initialization method for declaring objects of our class, we need a method for performing the forward propagation step. Complete the `forward_prop_routine` function. This will later be linked to the `forward_prop` routine of the `SimpleRNN` class. 

In [23]:
def forward_prop_routine(self, xs, reset_h=False):
    """
    Performs forward propagation of the input `xs` in sequential order, and then returns all predictions `ys_pred`
    The steps are: [Input x] -> [Hidden h] -> [Output unnormalized z] -> [Prediction probabilities y]

    Arguments:
    xs - a python list of one-hot characters to predict the next character from,
         Has a shape of list([1,OUTPUT_DIM]) and length `len(xs)`
    reset_h - boolean value, whether or not the hidden state should be reset

    Returns:
    ys_pred - a python list of prediction probabilities for each character in `xs`. There are `len(xs)`
              elements in ys_pred, each element `i` of shape (1, output_dim) and type `numpy.ndarray`
              that contains the probabilities of what the next character xs[i+1] can be

    Example:
    xs - [np.array([0,0,1]), np.array([0,1,0]), np.array([1,0,0])]
    ys_pred (ground truth answer) - [np.array([0,1,0]), np.array([1,0,0]), np.array([...])]
    """
    # initialize the list of predictions
    ys_pred = [0] * len(xs)

    # load the last hidden state of the previous batch
    if reset_h:
        self.cache['h_start'] = np.zeros((1, self.W_hh.shape[0]))
    hs = {-1: self.cache['h_start']}

    # forward propagate
    for index, item in enumerate(ys_pred):
        z = xs[index] @ self.W_xh + hs[index-1] @ self.W_hh + self.b_h
        h = np.tanh(z)
        y = h @ self.W_hy + self.b_y
        p = np.exp(y - np.max(y)) # softmax numerator
        p = p / np.sum(p)         # softmax
        ys_pred[index] = p
        hs[index] = h
    
    # save list of hidden state in cache to use later in backprop
    self.cache = {
        'hs': hs,
        'xs': xs,
        'h_start': hs[len(xs) - 1]
    }
    return ys_pred

The following cell tests if your implementation is correct (it assumes you correctly solved task 1.2.1):

In [24]:
test_forward_prop_routine(init_routine, forward_prop_routine)

test passed, dimensions are correct


#### 1.2.3 Backward propagation

Now that you developed methods for initialization and forward propagation, it's time to implement the backward propagation algorithm as a method for our class.

The cross-entropy loss will be used for this task, and just like in **IHA1**, backward pass of the softmax activation function and the cross-entropy loss can be combined into a simple formula: $\mathbf{\hat{p_t}} - \mathbf{p_t}$, where $\mathbf{\hat{p_t}}$ is the predicted output vector for time step $t$ and $\mathbf{p_t}$ is the ground truth one-hot vector for time step $t$.
The full collection of backward pass formulas that are required for implementing `backward_prop_routine` are shown in Equations 5-12:  

$$
\frac{d\mathcal{L}_t}{d \mathbf{y}_t}=\hat{\mathbf{p}}_t-\mathbf{p}_t \tag{5} ~,~ \text{ for } t=1,\dots,N, 
$$

$$
\frac{d\mathcal{L}}{d\mathbf{W}_{hy}}=\sum_{t=1}^N\mathbf{h}_t^T\frac{d\mathcal{L}_t}{d \mathbf{y}_t}, \tag{6}
$$

$$
\frac{d\mathcal{L}}{d\mathbf{b}_{y}}=\sum_{t=1}^N\frac{d\mathcal{L}_t}{d \mathbf{y}_t},   \tag{7}
$$

$$
\frac{d\mathcal{L}}{d\mathbf{h}_t}=\frac{d \mathcal{L}_t}{d \mathbf{y}_t}\mathbf{W}_{hy}^T+(1-\mathbf{h}_{t+1}^2)\odot\frac{d\mathcal{L}}{d\mathbf{h}_{t+1}}\mathbf{W}_{hh}^T   \tag{8.1} ~,~ \text{ for }t=1,\dots,N-1,
$$

$$
\frac{d\mathcal{L}}{d\mathbf{h}_N}=\frac{d \mathcal{L}_t}{d \mathbf{y}_N}\mathbf{W}_{hy}^T,   \tag{8.2}
$$

$$
\frac{d\mathcal{L}}{d\mathbf{z}_t}=(1-\mathbf{h}_t^2)\odot\frac{d\mathcal{L}}{d\mathbf{h}_t}   ~,~ \text{ for } t=1,\dots,N,\tag{9}
$$

$$
\frac{d\mathcal{L}}{d\mathbf{W}_{xh}}=\sum_{t=1}^N\mathbf{x}_t^T\frac{d\mathcal{L}}{d\mathbf{z}_t},   \tag{10}
$$

$$
\frac{d\mathcal{L}}{d\mathbf{W}_{hh}}=\sum_{t=1}^N\mathbf{h}_{t-1}^T\frac{d\mathcal{L}}{d\mathbf{z}_t},   \tag{11}
$$

$$
\frac{d\mathcal{L}}{d\mathbf{b}_h}=\sum_{t=1}^N\frac{d\mathcal{L}}{d\mathbf{z}_t},   \tag{12}
$$
where $\odot$ is the <a href="https://en.wikipedia.org/wiki/Hadamard_product_(matrices)">hadamard product / elementwise multiplication</a>.  

Complete the `backward_prop_routine` function below. This will later be linked to the `backward_prop` method of the `SimpleRNN` class.

In [26]:
def backward_prop_routine(self, ys, ys_pred):
    """
    Performs backward propagation, calculating the gradients of every trainable weight. The gradients should
    be clipped into the interval [-5,5]

    Arguments:
    ys - a python list of true labels (one-hot vectors of type `numpy.ndarray` for every character to predict)
         has a shape list( (1,OUTPUT_DIM) )
    ys_pred - a python list of predicted labels (one-hot vectors of type `numpy.ndarray` for every character predicted)
         has a shape list( (1,OUTPUT_DIM) )

    Returns:
    gradients - a dictionary of the gradients of every trainable weight. The keys are the weight variable names
                and the values are the actual gradient value
    """
    # extract from cache
    hs = self.cache['hs']
    xs = self.cache['xs']
    
    # intialize all variables
    N = len(ys)
    db_h = db_y = dW_hy = dW_xh = dW_hh = 0.0
    y_gradients = [ys_pred[t] - ys[t] for t in range(N)]
    
    # loop over all time steps to compute gradients w.r.t each hiddent state
    h_gradients = [y_gradients[t] @ self.W_hy.T for t in range(N)]
    for t in range(N-1, 0, -1):
        h_gradients[t-1] += (1 - hs[t] * hs[t]) * h_gradients[t] @ self.W_hh.T

    # loop over each time step to accumulate the gradients
    for t in range(N):
        dz_t = (1 - hs[t] * hs[t]) * h_gradients[t]
        db_h += dz_t
        db_y += y_gradients[t]
        dW_hy += hs[t].T @ y_gradients[t]
        dW_xh += xs[t].T @ dz_t
        dW_hh += hs[t-1].T @ dz_t
        
    # clip gradients to prevent explosion
    for dparam in [dW_xh, dW_hh, dW_hy, db_h, db_y]:
        np.clip(dparam, -5, 5, out=dparam)

    # save gradients into dict and return
    gradients = {
        'W_xh': dW_xh, 
        'W_hh': dW_hh, 
        'W_hy': dW_hy, 
        'b_h': db_h, 
        'b_y': db_y
    }
    return gradients

The following cell tests if your implementation is correct (it assumes you correctly solved tasks 1.2.1 and 1.2.2):

In [27]:
test_backward_prop_routine(init_routine, forward_prop_routine, backward_prop_routine)

Your BPTT implementation is correct.


#### 1.2.4 Applying the gradients

Finally, we need a method that updates the paramters of the RNN cell, given the computed gradients of the loss with respect to each one of them. Complete the `apply_gradients_routine` function below. This will later be linked to the `apply_gradients` method of the `SimpleRNN` class.

In [28]:
def apply_gradients_routine(self, gradients, learning_rate):
    """ Performs the weight update procedure. Updates every weight with its corresponding gradient 
    found in the `gradients` input dictionary

    Arguments:
    gradients - dictionary containing (key, value) pairs, where the key is a weight variable name of
                the `SimpleRNN` class and the value is the gradient to apply to the matching weight
                Example: {'W_xh':0.04, 'b_h':0.11}
    learning_rate - the learning rate to use for this iteration of weight updates
    """
    self.W_xh -= learning_rate * gradients['W_xh']
    self.W_hh -= learning_rate * gradients['W_hh']
    self.W_hy -= learning_rate * gradients['W_hy']
    self.b_h -= learning_rate * gradients['b_h']
    self.b_y -= learning_rate * gradients['b_y']

The following cell tests if your implementation is correct (it assumes you correctly solved task 1.2.1):

In [29]:
test_apply_gradients_routine(init_routine, apply_gradients_routine)

test apply gradients passed


#### 1.2.5 Putting everything together
Now that all the parts of the RNN are implemented, we can define the class. The following cell defines the `SimpleRNN` class, linking all the functions you developed in the earlier tasks to corresponding methods. Additionally, it also implements another method, the `sample`, used to generate output from a trained RNN. Note that you don't have to change anything in this cell, just run it after solving the previous tasks.

In [30]:
class SimpleRNN:
    """
    The simple RNN class definition
    Implements the initializations of the weights, the forward propagation from `x` to `y_pred`,
    the backward propagation with respect to each trainable weight and the update weights rule.
    """
    
    # __init__ method is now the initialization_routine function
    __init__ = init_routine
        
    # forward_prop method is the forward_prop_routine function
    forward_prop = forward_prop_routine
    
    # backward_prop method is the backward_prop_routine function
    backward_prop = backward_prop_routine
    
    # apply_gradients method is the apply_gradients_routine function
    apply_gradients = apply_gradients_routine


    def sample(self, seed, n, char_to_onehot, chars):
        """ Given a seed character `seed`, generates a sequence of `n` characters by continuously feeding the output
        character of the RNN at time t as the input for the next time step at time t+1. If the RNN is trained well, 
        this method should be able to generate sentences that resembles some kind of structure. 
        The character should be generated with a probability of the outputs of the network!
        
        Arguments:
        seed - a first character of type str to feed into the SimpleRNN
        n - the length of the character sequence to generate
        char_to_onehot - a dict that maps keys of characters to its one-hot representation. 
                         You created this dict earlier in the lab
        char - a list of chars (vocabulary). You also created this list earlier in the lab
        """
        
        saved_cache = copy.deepcopy(self.cache)
        
        char_list = []
        h = self.cache['h_start']

        # seed one-hot char
        x = char_to_onehot[seed]

        for t in range(n):
            """ not relevant anymore
            # perform forward prop. `forward_prop` is not called because then the hidden state for the
            # training procedure will be overwritten by the hidden state generates by this sampling procedure
            h = np.tanh(np.dot(x, self.W_xh) + np.dot(h, self.W_hh) + self.b_h)
            y = np.dot(h, self.W_hy) + self.b_y
            p = np.exp(y) / np.sum(np.exp(y))
            """
            p = self.forward_prop(x)[0]
            
            # randomly pick a character with respect to the probabilities of the output of the network
            ix = np.random.choice(range(self.input_dim), p=p.ravel())

            x = np.zeros((1, self.input_dim))   
            x[0,ix] = 1

            # save the generated character
            char_list.append(chars[ix])
            
        self.cache = saved_cache

        return ''.join(char_list)

### 1.3 Training the RNN <a class="anchor" id="1.3"></a>
In this section, code has been provided to you to train a `SimpleRNN` cell by predicting the next charcter in sequence.  

The code below has the following properties: 
 * Loop over `data`, using `seq_length` words at a time as a batch to train with
 * When `data` has been fully iterated through, start over again
 * Start over again with `data` in an infinite loop and then interrupt the running cell
 when results from `sample` are good enough.  
 * Perform `forward_prop`, `backward_prop` and  `apply_gradients` with each batch of characters
 * every 1000th iteration:
     * compute the cross-entropy loss of the current batch
     * sample a character sequence of length 200
     * print the loss and the sampled sequence

In [ ]:
rnn = SimpleRNN(len(chars), 500, len(chars))

# the number of characters to perform one update of the network with,
# can be thought of as a batch
seq_length = 25

# the learning rate to use
learning_rate = 0.001

while True: # the infinite loop
    
    # iterate over the data set
    for i in range(len(data)//seq_length):
        
        # convert to one-hot
        xs = [char_to_onehot[c] for c in data[i*seq_length:(i+1)*seq_length]]#inputs to the RNN
        ys = [char_to_onehot[c] for c in data[i*seq_length+1:(i+1)*seq_length+1]]#the targets it should be outputting

        # fp, bp and update
        ys_pred = rnn.forward_prop(xs)
        gradients = rnn.backward_prop(ys, ys_pred)
        rnn.apply_gradients(gradients, learning_rate)

        # print loss and sample a sentence every 1000th iteration
        if i%1000==0:
            loss = np.sum([ -1.0 *np.sum(y * np.log(y_pred + 1e-8)) for y, y_pred in zip(ys, ys_pred) ]) # x-entropy
            print('iteration %d, loss: %f' % (i, loss))
            text = rnn.sample('a', 200, char_to_onehot, chars)
            print(text, end='\n\n')

Run the following cell to produce sentences using the trained RNN:

In [32]:
# kafka, including upper chars
for i in range(10):
    print('------ Sentence %i: ------' %i)
    print(rnn.sample('a', 200, char_to_onehot, chars))

------ Sentence 0: ------
s car hn oshs wsis sbeed borl isipHtm not edsed r lo gihuc hr eo we une  wnt uoe Hloo whq aalre honyhp tntess tolrs onmsrzB wh hih gieerd ufims frgnor gae olnHwnrnbd ale har tor atu ibd xatduce wrm co
------ Sentence 1: ------
 ;aen hily hhvuam re lnos iyeb pa, bae sr wo wyr wo in gi rrusn agsilo jhr me ,oly ee gk lea swod get ig hresitlk ool ld inm hnuFt nr te moa anunte wre ehn tonne no htmlhslawe, do) chs ptnp wunG twir 
------ Sentence 2: ------
l wo bjiws ohy rhbr aodm tvrp wht ossrnc nbt el  ta' fonoa fhlnswlexrewmni w,oBa. cae oblid Yor,e hest) eys ond pet ,e ttsh xd worugegctl lonoi bi pas wms, hie yh e nmitn fosrr mewa cto xa wnu se hav 
------ Sentence 3: ------
ir tolu Iil tonom tr  nr ddtl te, ahe oo Or,d pht  aomn, bole nh t rs heeasi sysrtho Mosr tod gier thaty zkwags storw. cle ho d, win lde ,leo,nd uwd fhla aeC hnr ,o nosl, oimaee tas iity eh" wae chf a
------ Sentence 4: ------
iy s Ue meu hob ir yo, drrt ,i porooems tith fat pmd sf wauasss to

Here are some generated sentences for reference how good the results should be. Not quite passing any spell checking program, but there are some real english words in there and the sentences seem to have a reasonable structure.

    ------ Sentence 0: ------
    ted eat aread noup, seed, but. ythed whithit tued that and in ask chemr tirme ble serpareing. he wionly memind. Dedteren drager, chat ther se stealeay. The kpay shir in wha dangereag he torearr ieve w
    ------ Sentence 1: ------
    swayly wase Mlamer io beell hom nofrreang, hr meed tould, Grere, werned coull gitr theQr coule verustengs at jeas whear', fasmer's, ad ho mofello goin "hey in normablite chas a) he. Heenew "thel seefr
    ------ Sentence 2: ------
    r's avethen harly dsundiigex; thinew, oug lertinestad to the wourd jhen sorywe, thever shene It, thour" hawss He:thon's batiar's sfin, luike he. Ar uver. Af. qanilew ucheed attricgwaynong roked t- an 
    ------ Sentence 3: ------
    tor
     unde, thab haw dtidtly kad sendeder, ithed far theme, veedlly nowlyin; and )rece thete disse, out louthe, row the rapeasterse at. Heed rnweem. "ut he bleitny Grerorot ie reryibe. So in., Hers ane
    ------ Sentence 4: ------
    ding, fitse. . Whisly andit, she haing "ya bistatily dlatkedor whing saded theust, but the stere stotady is eleaprcentake, got to mame luch ally, mus llablly, ach caling, at what, he want aar athen d'
    ------ Sentence 5: ------
    cll"end sat what herpingoth thouthar ald beinvaver.

    Tere sreaned on in she was athar thatssu fres is at lees morther all sroaven to somichary mothtrey norytiot a tior sow bercead lassedore towist at 
    ------ Sentence 6: ------
    gly.

    Shen in shend comrie t oreth ubserwer he dist in commicherr. Wrmeathithemy nougracl.y'e sate horkllomisted betrnowhrous wfverthr bleatekeM tusinitedy Herach mop wislrocly theirse
     on illy ans in
    ------ Sentence 7: ------
    dd, puting ot been ay theed . Hemula, he farer". The h hanor, and dandid to the sedertereveng untint.

    Wken", surme ceared thet lomp io wotcouUir onsur"ont ar with, thancMmperny him., is sas bees coul
    ------ Sentence 8: ------
    d natmen", he theretotist wist, Gregely, arly re caule he praser, and woll buccem, then atse. Hemu haded ditince and dleyrist comply. "Vm inen beglings wet ou't dot, plest willay theren lipsest inpren
    ------ Sentence 9: ------
    byed Gregst oonsbecemed thet ad herladble, din ther, and at. Numferringtice st ead thelr way mors. Ium loomul themo head roon inf thet the. "lencorn was ned in tuc'e foot butimed, -feritn be thand Yun

You may try different learning rates, different number of neurons for the hidden state and try to prune the text from special characters and upper characters, but the fact is that this is only one single RNN cell and it is as good as it gets.  

## 2 Neural Machine Translation <a class="anchor" id="2"></a>

In this task you will implement a small neural machine translator using the Keras API. The final model will be able to translate english sentences into spanish sentences. Unlike the previous task, the input and output at each time step will now represent an entire word instead of a character.  

The task is divided into two main sections:
 * Pre processing of data
 * Building a sequence-to-sequence RNN
 
The principle of sequence-to-sequence modelling is shown in the figure below
![seq2seq unrolled](utils/images/seq2seq.png)
In your case, an english sentence is input to the encoder network, one word at each time step. After the sentence has been fed, there will now be a representation of the entire sentence encoded into the hidden state of the RNN. The hidden state of the decoder network is thereafter initialized with this representation and tries to generate the sequence of spanish words from the initial hidden state.  

Run the cell below to import the necessary packages to get started with the task

In [1]:
import re
import numpy as np
import unidecode
from keras.preprocessing.sequence import pad_sequences
from utils.tests.ha2Tests import *

Using TensorFlow backend.


### 2.1 Pre-processing <a class="anchor" id="2.1"></a>
The dataset to use consists of bilingual sentence pairs. Each sentence in the list of english sentences has a corresponding spanish translation at the same index in the list of spanish sentences.  

Basic preprocessing has already been performed such as:  
 * to lower case
 * remove any padded spaces at the start and end of the row (trim)
 * convert from unicode character set to the ASCII character set. Example: á --> a and ñ --> n.
 * If the character is a question mark, punctuation or exclamation mark: ? ! ., put a space to the left of the character to make the char into a separate word.
 * remove any non-letter character
 * split each line into a tuple, where the left element is the source language sentence and the right element is the target language sentence
 * remove any sentence pair with number of words more than 5

#### 2.1.1 Loading and inspecting the data set <a class="anchor" id="2.1.1"></a>
Lets first load the english-spanish sentence pairs into memory and inspect some samples.

In [2]:
# load data set
lines = open('utils/spa.txt', 'r', encoding='UTF-8').read().strip().split('\n')
data = pickle.load( open( "utils/spa-preprocessed.pkl", "rb" ) )
spa, eng = data['spa'], data['eng']
SEQ_MAX_LEN = 5

print('There are %i sentence pairs' % len(lines))
print('The shortest english sentence is %i words' 
      % np.min(list(map(lambda line: len(line.split(' ')), eng))))
print('The shortest spanish sentence is %i words' 
      % np.min(list(map(lambda line: len(line.split(' ')), spa))))
print('The longest english sentence is %i words' 
      % np.max(list(map(lambda line: len(line.split(' ')), eng))))
print('The longest spanish sentence is %i words' 
      % np.max(list(map(lambda line: len(line.split(' ')), spa))))
print('\n10 Random short sentence pairs:')
for i in range(10):
    ix = np.random.choice(int(len(eng)))
    print(eng[ix],'\t',spa[ix])

There are 114282 sentence pairs
The shortest english sentence is 2 words
The shortest spanish sentence is 2 words
The longest english sentence is 5 words
The longest spanish sentence is 5 words

10 Random short sentence pairs:
ill explain everything . 	 lo explicare todo .
this is my room . 	 este es mi cuarto .
the criminal is nervous . 	 el criminal esta nervioso .
i dont see it . 	 no lo veo .
who speaks french ? 	 quien habla frances ?
this is mine . 	 este es el mio .
well go together . 	 vamos a ir juntos .
i couldnt walk . 	 no podia caminar .
tom needs a doctor . 	 tom necesita un medico .
they stayed friends . 	 ellos siguieron siendo amigos .


#### 2.1.4 Word-to-index and index-to-word conversions <a class="anchor" id="2.1.4"></a>
Just like in Task 1 earlier, you need dictionaries to transform between (in this case) the words and unique number representations. The following dictionaries/lists are implemented:  

 * `eng_ix_to_word` - a python list where each index is a unique number that represents the english word stored as value.
                      `eng_ix_to_word[0]` should equal to `ZERO`
                      `enx_ix_to_word[-1]` (the last element) should equal to `UNK` (unknown)
 * `eng_word_to_ix` - like `eng_ix_to_word` but reversed, a dictionary mapping every word into the unique number. The key is a word that also exists in `eng_ix_to_word` and the value is an integer, that matches the index of the word in `eng_ix_to_word`
 * `spa_ix_to_word` - same principle as `eng_ix_to_word`, but from the vocabulary of `spa`
                      `spa_ix_to_word[0]` should equal to `ZERO`
                      `spa_ix_to_word[-1]` (the last element) should equal to `UNK` (unknown)
 * `spa_word_to_ix` - same principle as `eng_word_to_ix`, but from the vocabulary of `spa`

In [3]:
eng_vocab = set([word for sentence in eng for word in sentence.split(' ')])
eng_ix_to_word = ['ZERO'] + list(eng_vocab) + ['UNK']
eng_word_to_ix =  { word: index for index, word in enumerate(eng_ix_to_word)}
spa_vocab = set([word for sentence in spa for word in sentence.split(' ')])
spa_ix_to_word = ['ZERO'] + list(spa_vocab) + ['UNK']
spa_word_to_ix =  { word: index for index, word in enumerate(spa_ix_to_word)}

Now we convert the words of `eng` and `spa` into their corresponding index numbers  

Run the cell below to initialize these variables:  
 * X - A list of sentences. Each sentence is represented by a list where the i'th element is the index number representing the i'th word in the sentence string.
 * Y - same principle as `X`, but contains the converted spanish sentences  
 
 Example:
 if `eng_word_to_ix` is defined as `{'good':0, 'morning':1}`  
 and `eng` is defined as `["good morning", "morning"]`  
 then `X` should result in `[[0,1],[1]]`  

In [4]:
X = [ [eng_word_to_ix[word] for word in sentence.split(' ')] for sentence in eng]
Y = [ [spa_word_to_ix[word] for word in sentence.split(' ')] for sentence in spa]

#### 2.1.5 Padding <a class="anchor" id="2.1.5"></a>
Since the sentences are of variable lengths, padding is needed to make every sentence equal length. Every sentence list in `X` and `Y` are padded with zeros to make every list equal to the length of the maximum sentence length. The zeros are **prepended** before the first word.  

Example:  
if `X` is defined as `[[1,2,3], [1,2], [1]]`   
then after applying padding, X is equal to `[[1,2,3], [0,1,2], [0,0,1]]`  

In [5]:
X = pad_sequences(X, maxlen=SEQ_MAX_LEN, dtype='int32')
Y = pad_sequences(Y, maxlen=SEQ_MAX_LEN, dtype='int32')

#### 2.1.6 One-hot labels <a class="anchor" id="2.1.6"></a>
The last pre processing converting the data into one-hot vectors.  

For this task, **only** the labels (spanish words) are going to be transformed into one-hot vectors. The data points of english words are only going to be transformed into their corresponding numeric values by `eng_word_to_ix` because `Embedding` from the keras API already implements converting the inputs to one-hot vectors.  

Transform `Y` to have the following properties:
 * define `Y` as a numpy matrix of shape (number of sentences, number of words of longest sentence, one-hot vector length). 
 * dimension 0 represents each sentence (batch).
 * dimension 1 represents each word in a sentece (padded with zeros for equal length)
 * dimension 2 represents the one-hot vector for that particular word in that particular sentence. To pass the test, a onehot representation of word `1023` should be a zero vector with its 1023th index set to 1

In [6]:
# TODO: convert each index number in Y into its corresponding one-hot vector
num_output = len(spa_word_to_ix)
print("Tip: the shape of Y should be: (%i, %i, %i)" % (len(Y), len(Y[0]), num_output))

def Y_to_onehot(Y, num_output):
    # Complete function according to description above
    # YOUR CODE HERE
    from keras.utils import to_categorical
    Y_ = []
    for y in Y:
        encoded = to_categorical(y, num_output)
        Y_.append(encoded)
    return np.array(Y_)

Tip: the shape of Y should be: (22947, 5, 8805)


Run the following cell to test your implementation.

In [7]:
# test case 
test_Y_to_onehot(Y_to_onehot)

test passed


When you passed the test, run the cell below to convert the sentences of Y into lists of onehot vectors

In [8]:
Y_onehot = Y_to_onehot(Y, num_output)

### 2.2 Implementing a sequence-to-sequence model <a class="anchor" id="2.2"></a>
In this section you will define, train and evaluate a sequence-to-sequence RNN architecture with the help of the Keras API.  

#### 2.2.1 Defining the architecture <a class="anchor" id="2.2.1"></a>

Now you are going to define the architecture for the translator. There are several possible architectures for performing translation, which is a sequence-to-sequence task (an input sequence, the English sentence, mapped to an output sequence, the Spanish sentence). If you take inspiration from searching the internet, you might come across neural machine translators such as [Google's Neural Machine Translation System](http://tsong.me/blog/google-nmt/). This kind of deep architecture was trained for a week using 100 GPUs, which is completely out of reach for the purposes of this assignment. Instead we will ask you to implement a simpler architecture that, even though it doesn't possess all the important parts of a translation network, still performs acceptably.

The architecture you are required to implement will closely resemble the image shown at the start of this task. First the input sequence is fed to an encoder network, which is in charge of summarizing the entire sequence in its internal memory. This internal memory is then fed to a decoder network, which uses this summarized content to produce the output sequence. Both the encoder and the decoder networks will be comprised of a single LSTM unit, with a memory cell of arbitrary dimension. Since we won't stack LSTM layers, this can be seen as a "toy" version of a complete LSTM architecure.

#### Practical remarks

Note that the input sequence for each training sentence is a vector with 5 elements, where each element corresponds to the number of that particular word in the vocabulary. 

Because similar word indexes don't correspond to similar words (e.g. index 133 corresponds to 'joke', whereas index 134 corresponds to 'silence'), it's easier to train our network if we map this data to a space where the semantic meaning of the word is related to its geometric position. For instance, we would like the word 'car' to be closer to the word 'bus' than it is to 'tree'. This can be accomplished with the `Embedding` layer module from Keras, which tries to find a suitable representation for positive integers in a higher-dimensional space by using dimensionality reduction techniques in the matrix of co-ocurrence statistics of our input data. This layer can use an already trained mapping (like Word2Vec), but in this exercise we'll train it ourselves.

Besides deciding how to encode the input, there is another important practical remark to be done. Since the decoder network will also be implemented as an LSTM, it will require inputs. According to what was described earlier, the only important data for this network is the memory cell at each time-step. Unfortunately, there is no simple way to create an LSTM cell without inputs (you can, of course, customize the LSTM layer to not receive any inputs), so we'll simply feed zeros as input at each time-step and it's likely that the optimization procedure will tune the decoder weights to disregard this input eventually (because it's completely uncorrelated to the input and the label, so it doesn't help with the prediction at all).

#### Keras tips

- It's necessary to use the [Model class API](https://keras.io/models/model/) for implementing this model (because it has two inputs, the input sequence and the zeros to the decoder). Be sure to familiarize yourself with this way of defining models in Keras.
- You can define an LSTM cell using the [LSTM layer](https://keras.io/layers/recurrent/#lstm).
- The LSTM layer by default outputs only the output for the last time-step, which is great for the encoder part, since we don't care about its outputs anyway. However, for the decoder network we want the outputs at all time steps. To accomplish this, you can set the argument `return_sequences` to `True` when creating the LSTM layer.
- You will need to feed the memory cell content of the encoder network at the last time-step to the decoder network. You can ask the LSTM layer to also output its memory cell contents by setting the `return_state` argument to `True` when creating the layer (this makes the LSTM layer output three things: the actual LSTM output and two variables with different parts of the LSTM internal memory cell).
- You can specify the initial state of LSTM layers symbolically by calling them with the keyword argument  `initial_state`. The value of `initial_state` should be a tensor or list of tensors representing the initial state of the LSTM layer.


Now, implement your sequence-to-sequence architecture in the function `create_model` by following the descriptions of the function.  

In [11]:
# Add your import statements here
from keras import Input, Model
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, Embedding
from keras.layers.recurrent import LSTM

def create_model(input_n, X_seq_len, output_n, Y_seq_len, hidden_dim, embedding_dim):
    """ Define a keras sequence-to-sequence model. 
    
    Arguments:
    input_n - integer, the number of inputs for the network (the length of a one-hot vector from `X`)
    X_seq_len - integer, the length of a sequence from `X`. Should be constant and you made sure by using padding
    output_n - integer, the number of outputs for the network (the length of a one-hot vector from `Y`)
    Y_seq_len - integer, the length of a sequence from `Y`. Should be constant and you made sure by using padding
    hidden_dim - integer, number of units in the LSTM's memory cell.
    embedding_dim - output dimension of the embedding layer.
    
    Returns:
    The compiled keras model
    
    """
    # Input and embedding layers
    encoder_input_layer = Input(shape=[X_seq_len])
    encoder_embedding_layer = Embedding(input_n, embedding_dim, input_length=X_seq_len, mask_zero=True)(encoder_input_layer)
    
    # Create the encoder LSTM.
    # YOUR CODE HERE
    from keras import regularizers
    encoder_outputs, state_h, state_c  = LSTM(hidden_dim, return_state=True)(encoder_embedding_layer)
    encoder_states = [state_h, state_c]
    
    # Null input (should be fed with zeros) and repeating it the same number of times as there are words in the 
    # target sentence
    null_input = Input(shape=[1])
    repeated_null = RepeatVector(Y_seq_len)(null_input)
    
    # Create the decoder LSTM (feed it with the memory of the encoder network). Call it `decoder_layer`
    # YOUR CODE HERE
    decoder_layer, _, _ = LSTM(hidden_dim, return_sequences=True, return_state=True)(repeated_null,initial_state=encoder_states)
    
    # Add a fully connected layer and a softmax to the outputs of the decoder
    decoder_fully_connected = TimeDistributed(Dense(output_n))(decoder_layer)
    decoder_softmax = Activation('softmax')(decoder_fully_connected)
    
    # Create final model and compile it
    model = Model([encoder_input_layer, null_input], decoder_softmax)
    
    # Compile the model. Use a loss function, optimizer, and metrics of your choice
    # YOUR CODE HERE
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Add these arguments to the model for convenience
    model.hidden_dim = hidden_dim
    model.embedding_dim = embedding_dim
    
    return model

#### 2.2.2 Training the model <a class="anchor" id="2.2.2"></a>
Now create the model and train it. Try out different hyper-parameters if you're not satisfied with the result. For obtaining a good speedup using the GPU, opt for large number of memory cells and large batch sizes (although not too large, that also has drawbacks).

In [17]:
# Computing inputs for `create_model`
input_n = len(eng_word_to_ix)
output_n = len(spa_word_to_ix)
X_seq_len = len(X[0])
Y_seq_len = Y_onehot.shape[1]

# Create the model
# YOUR CODE HERE
model = create_model(input_n, X_seq_len, output_n, Y_seq_len, 256, 512)

# Train the model (note that you should feed both the input sentence and a zero array with the correct shape to
# the `fit` method).
# YOUR CODE HERE
from keras.callbacks import ModelCheckpoint, EarlyStopping
mc = ModelCheckpoint('./models/best_model.h5', monitor='val_loss', save_best_only=True)
estop = EarlyStopping(monitor='val_acc', patience=3)
model.fit(x=[X, np.zeros(X.shape[0])], y=Y_onehot, epochs=200, batch_size=256, validation_split=0.2, callbacks=[mc, estop])
model.save_weights('seq2seq_model_correct.hdf5')

# If you need, you can use the following line for loading a saved model weights
#model.load_weights('seq2seq_model.hdf5')

Train on 18357 samples, validate on 4590 samples
Epoch 1/200
18357/18357 [==============================] - ETA: 3:35 - loss: 9.0829 - acc: 0.0000e+0 - ETA: 1:50 - loss: 9.0816 - acc: 0.0996    - ETA: 1:15 - loss: 9.0801 - acc: 0.170 - ETA: 58s - loss: 9.0782 - acc: 0.212 - ETA: 47s - loss: 9.0761 - acc: 0.23 - ETA: 40s - loss: 9.0734 - acc: 0.24 - ETA: 35s - loss: 9.0699 - acc: 0.25 - ETA: 31s - loss: 9.0655 - acc: 0.26 - ETA: 28s - loss: 9.0590 - acc: 0.26 - ETA: 26s - loss: 9.0503 - acc: 0.27 - ETA: 24s - loss: 9.0379 - acc: 0.28 - ETA: 22s - loss: 9.0203 - acc: 0.28 - ETA: 21s - loss: 8.9962 - acc: 0.28 - ETA: 19s - loss: 8.9625 - acc: 0.29 - ETA: 18s - loss: 8.9162 - acc: 0.29 - ETA: 17s - loss: 8.8546 - acc: 0.29 - ETA: 16s - loss: 8.7761 - acc: 0.29 - ETA: 16s - loss: 8.6847 - acc: 0.29 - ETA: 15s - loss: 8.5823 - acc: 0.29 - ETA: 14s - loss: 8.4753 - acc: 0.29 - ETA: 13s - loss: 8.3631 - acc: 0.29 - ETA: 13s - loss: 8.2508 - acc: 0.30 - ETA: 12s - loss: 8.1404 - acc: 0.30 - ETA

c:\temp\vm\dmlgpu\lib\site-packages\keras\engine\network.py:872: UserWarning: Layer lstm_12 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_11/while/Exit_3:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_11/while/Exit_4:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/200
18357/18357 [==============================] - ETA: 8s - loss: 5.0672 - acc: 0.324 - ETA: 8s - loss: 5.1004 - acc: 0.318 - ETA: 8s - loss: 5.0883 - acc: 0.316 - ETA: 8s - loss: 5.0726 - acc: 0.314 - ETA: 8s - loss: 5.0673 - acc: 0.313 - ETA: 8s - loss: 5.0582 - acc: 0.315 - ETA: 8s - loss: 5.0536 - acc: 0.315 - ETA: 7s - loss: 5.0614 - acc: 0.314 - ETA: 7s - loss: 5.0538 - acc: 0.316 - ETA: 7s - loss: 5.0461 - acc: 0.317 - ETA: 7s - loss: 5.0500 - acc: 0.317 - ETA: 7s - loss: 5.0429 - acc: 0.318 - ETA: 7s - loss: 5.0424 - acc: 0.319 - ETA: 7s - loss: 5.0484 - acc: 0.318 - ETA: 7s - loss: 5.0474 - acc: 0.318 - ETA: 6s - loss: 5.0434 - acc: 0.318 - ETA: 6s - loss: 5.0443 - acc: 0.319 - ETA: 6s - loss: 5.0415 - acc: 0.319 - ETA: 6s - loss: 5.0351 - acc: 0.319 - ETA: 6s - loss: 5.0338 - acc: 0.318 - ETA: 6s - loss: 5.0332 - acc: 0.319 - ETA: 6s - loss: 5.0319 - acc: 0.319 - ETA: 6s - loss: 5.0273 - acc: 0.320 - ETA: 5s - loss: 5.0250 - acc: 0.321 - ETA: 5s - loss: 5.0205 - acc:

18357/18357 [==============================] - ETA: 8s - loss: 4.6859 - acc: 0.332 - ETA: 8s - loss: 4.6980 - acc: 0.323 - ETA: 8s - loss: 4.7047 - acc: 0.322 - ETA: 8s - loss: 4.6851 - acc: 0.324 - ETA: 8s - loss: 4.6702 - acc: 0.328 - ETA: 8s - loss: 4.6764 - acc: 0.327 - ETA: 8s - loss: 4.6858 - acc: 0.326 - ETA: 8s - loss: 4.6778 - acc: 0.327 - ETA: 8s - loss: 4.6784 - acc: 0.326 - ETA: 7s - loss: 4.6707 - acc: 0.327 - ETA: 7s - loss: 4.6724 - acc: 0.325 - ETA: 7s - loss: 4.6742 - acc: 0.323 - ETA: 7s - loss: 4.6767 - acc: 0.323 - ETA: 7s - loss: 4.6802 - acc: 0.321 - ETA: 7s - loss: 4.6745 - acc: 0.322 - ETA: 7s - loss: 4.6756 - acc: 0.323 - ETA: 6s - loss: 4.6684 - acc: 0.323 - ETA: 6s - loss: 4.6609 - acc: 0.325 - ETA: 6s - loss: 4.6665 - acc: 0.325 - ETA: 6s - loss: 4.6680 - acc: 0.324 - ETA: 6s - loss: 4.6707 - acc: 0.325 - ETA: 6s - loss: 4.6712 - acc: 0.324 - ETA: 6s - loss: 4.6657 - acc: 0.326 - ETA: 6s - loss: 4.6664 - acc: 0.327 - ETA: 5s - loss: 4.6655 - acc: 0.327 - ETA

18357/18357 [==============================] - ETA: 8s - loss: 4.3994 - acc: 0.357 - ETA: 8s - loss: 4.4276 - acc: 0.337 - ETA: 8s - loss: 4.4120 - acc: 0.340 - ETA: 8s - loss: 4.3983 - acc: 0.341 - ETA: 8s - loss: 4.4032 - acc: 0.343 - ETA: 8s - loss: 4.3991 - acc: 0.345 - ETA: 8s - loss: 4.3931 - acc: 0.347 - ETA: 8s - loss: 4.3801 - acc: 0.349 - ETA: 7s - loss: 4.3749 - acc: 0.348 - ETA: 7s - loss: 4.3673 - acc: 0.350 - ETA: 7s - loss: 4.3637 - acc: 0.350 - ETA: 7s - loss: 4.3624 - acc: 0.351 - ETA: 7s - loss: 4.3587 - acc: 0.350 - ETA: 7s - loss: 4.3639 - acc: 0.349 - ETA: 7s - loss: 4.3591 - acc: 0.350 - ETA: 7s - loss: 4.3632 - acc: 0.349 - ETA: 6s - loss: 4.3637 - acc: 0.348 - ETA: 6s - loss: 4.3637 - acc: 0.348 - ETA: 6s - loss: 4.3583 - acc: 0.349 - ETA: 6s - loss: 4.3524 - acc: 0.349 - ETA: 6s - loss: 4.3525 - acc: 0.349 - ETA: 6s - loss: 4.3527 - acc: 0.349 - ETA: 6s - loss: 4.3538 - acc: 0.348 - ETA: 5s - loss: 4.3538 - acc: 0.349 - ETA: 5s - loss: 4.3529 - acc: 0.349 - ETA

...

18357/18357 [==============================] - ETA: 8s - loss: 0.4623 - acc: 0.895 - ETA: 8s - loss: 0.4710 - acc: 0.894 - ETA: 8s - loss: 0.4701 - acc: 0.896 - ETA: 8s - loss: 0.4852 - acc: 0.893 - ETA: 8s - loss: 0.4951 - acc: 0.892 - ETA: 8s - loss: 0.4958 - acc: 0.892 - ETA: 8s - loss: 0.4955 - acc: 0.892 - ETA: 7s - loss: 0.4980 - acc: 0.890 - ETA: 7s - loss: 0.5049 - acc: 0.888 - ETA: 7s - loss: 0.5057 - acc: 0.888 - ETA: 7s - loss: 0.5009 - acc: 0.889 - ETA: 7s - loss: 0.5001 - acc: 0.889 - ETA: 7s - loss: 0.4979 - acc: 0.890 - ETA: 7s - loss: 0.5000 - acc: 0.890 - ETA: 7s - loss: 0.5013 - acc: 0.889 - ETA: 6s - loss: 0.5030 - acc: 0.888 - ETA: 6s - loss: 0.5038 - acc: 0.888 - ETA: 6s - loss: 0.5059 - acc: 0.887 - ETA: 6s - loss: 0.5052 - acc: 0.888 - ETA: 6s - loss: 0.5048 - acc: 0.887 - ETA: 6s - loss: 0.5059 - acc: 0.887 - ETA: 6s - loss: 0.5070 - acc: 0.886 - ETA: 6s - loss: 0.5081 - acc: 0.886 - ETA: 5s - loss: 0.5079 - acc: 0.886 - ETA: 5s - loss: 0.5065 - acc: 0.887 - ETA

18357/18357 [==============================] - ETA: 8s - loss: 0.4036 - acc: 0.909 - ETA: 8s - loss: 0.4004 - acc: 0.909 - ETA: 8s - loss: 0.4087 - acc: 0.906 - ETA: 8s - loss: 0.4293 - acc: 0.901 - ETA: 8s - loss: 0.4331 - acc: 0.898 - ETA: 8s - loss: 0.4364 - acc: 0.897 - ETA: 8s - loss: 0.4474 - acc: 0.894 - ETA: 8s - loss: 0.4483 - acc: 0.893 - ETA: 7s - loss: 0.4479 - acc: 0.894 - ETA: 7s - loss: 0.4549 - acc: 0.892 - ETA: 7s - loss: 0.4591 - acc: 0.890 - ETA: 7s - loss: 0.4554 - acc: 0.891 - ETA: 7s - loss: 0.4538 - acc: 0.891 - ETA: 7s - loss: 0.4544 - acc: 0.891 - ETA: 7s - loss: 0.4562 - acc: 0.891 - ETA: 6s - loss: 0.4574 - acc: 0.891 - ETA: 6s - loss: 0.4565 - acc: 0.891 - ETA: 6s - loss: 0.4585 - acc: 0.890 - ETA: 6s - loss: 0.4597 - acc: 0.890 - ETA: 6s - loss: 0.4567 - acc: 0.891 - ETA: 6s - loss: 0.4576 - acc: 0.891 - ETA: 6s - loss: 0.4558 - acc: 0.892 - ETA: 6s - loss: 0.4577 - acc: 0.891 - ETA: 5s - loss: 0.4575 - acc: 0.891 - ETA: 5s - loss: 0.4568 - acc: 0.891 - ETA

18357/18357 [==============================] - ETA: 8s - loss: 0.3944 - acc: 0.906 - ETA: 8s - loss: 0.3859 - acc: 0.912 - ETA: 8s - loss: 0.3766 - acc: 0.916 - ETA: 8s - loss: 0.3839 - acc: 0.913 - ETA: 8s - loss: 0.3860 - acc: 0.914 - ETA: 8s - loss: 0.3849 - acc: 0.914 - ETA: 8s - loss: 0.3872 - acc: 0.913 - ETA: 8s - loss: 0.3917 - acc: 0.911 - ETA: 7s - loss: 0.3970 - acc: 0.909 - ETA: 7s - loss: 0.3984 - acc: 0.908 - ETA: 7s - loss: 0.4007 - acc: 0.907 - ETA: 7s - loss: 0.4005 - acc: 0.906 - ETA: 7s - loss: 0.4000 - acc: 0.905 - ETA: 7s - loss: 0.3993 - acc: 0.905 - ETA: 7s - loss: 0.4022 - acc: 0.905 - ETA: 7s - loss: 0.4024 - acc: 0.905 - ETA: 6s - loss: 0.4027 - acc: 0.904 - ETA: 6s - loss: 0.4032 - acc: 0.904 - ETA: 6s - loss: 0.4028 - acc: 0.904 - ETA: 6s - loss: 0.4021 - acc: 0.904 - ETA: 6s - loss: 0.4032 - acc: 0.904 - ETA: 6s - loss: 0.4069 - acc: 0.903 - ETA: 6s - loss: 0.4062 - acc: 0.903 - ETA: 5s - loss: 0.4058 - acc: 0.903 - ETA: 5s - loss: 0.4049 - acc: 0.903 - ETA

18357/18357 [==============================] - ETA: 8s - loss: 0.3671 - acc: 0.906 - ETA: 8s - loss: 0.3789 - acc: 0.902 - ETA: 8s - loss: 0.3654 - acc: 0.906 - ETA: 8s - loss: 0.3592 - acc: 0.911 - ETA: 8s - loss: 0.3592 - acc: 0.912 - ETA: 8s - loss: 0.3606 - acc: 0.914 - ETA: 8s - loss: 0.3582 - acc: 0.915 - ETA: 8s - loss: 0.3611 - acc: 0.913 - ETA: 8s - loss: 0.3641 - acc: 0.912 - ETA: 7s - loss: 0.3642 - acc: 0.912 - ETA: 7s - loss: 0.3629 - acc: 0.912 - ETA: 7s - loss: 0.3647 - acc: 0.911 - ETA: 7s - loss: 0.3610 - acc: 0.912 - ETA: 7s - loss: 0.3588 - acc: 0.913 - ETA: 7s - loss: 0.3615 - acc: 0.913 - ETA: 7s - loss: 0.3645 - acc: 0.912 - ETA: 6s - loss: 0.3639 - acc: 0.912 - ETA: 6s - loss: 0.3631 - acc: 0.912 - ETA: 6s - loss: 0.3624 - acc: 0.912 - ETA: 6s - loss: 0.3630 - acc: 0.911 - ETA: 6s - loss: 0.3633 - acc: 0.911 - ETA: 6s - loss: 0.3654 - acc: 0.910 - ETA: 6s - loss: 0.3630 - acc: 0.910 - ETA: 6s - loss: 0.3622 - acc: 0.911 - ETA: 5s - loss: 0.3642 - acc: 0.910 - ETA

18357/18357 [==============================] - ETA: 9s - loss: 0.3326 - acc: 0.908 - ETA: 8s - loss: 0.3403 - acc: 0.914 - ETA: 8s - loss: 0.3352 - acc: 0.914 - ETA: 8s - loss: 0.3316 - acc: 0.916 - ETA: 8s - loss: 0.3335 - acc: 0.915 - ETA: 8s - loss: 0.3345 - acc: 0.915 - ETA: 8s - loss: 0.3409 - acc: 0.913 - ETA: 8s - loss: 0.3387 - acc: 0.914 - ETA: 7s - loss: 0.3369 - acc: 0.915 - ETA: 7s - loss: 0.3393 - acc: 0.915 - ETA: 7s - loss: 0.3360 - acc: 0.915 - ETA: 7s - loss: 0.3337 - acc: 0.916 - ETA: 7s - loss: 0.3317 - acc: 0.916 - ETA: 7s - loss: 0.3337 - acc: 0.915 - ETA: 7s - loss: 0.3334 - acc: 0.916 - ETA: 6s - loss: 0.3316 - acc: 0.917 - ETA: 6s - loss: 0.3294 - acc: 0.917 - ETA: 6s - loss: 0.3305 - acc: 0.916 - ETA: 6s - loss: 0.3334 - acc: 0.915 - ETA: 6s - loss: 0.3328 - acc: 0.915 - ETA: 6s - loss: 0.3310 - acc: 0.916 - ETA: 6s - loss: 0.3321 - acc: 0.915 - ETA: 6s - loss: 0.3324 - acc: 0.915 - ETA: 6s - loss: 0.3313 - acc: 0.916 - ETA: 5s - loss: 0.3309 - acc: 0.916 - ETA

#### 2.2.3 Evaluation <a class="anchor" id="2.2.3"></a>
You are free to evaluate and compare results of your model(s) in any way you have learned from the deep learning course.  

Now motivate your choice of architecture and hyperparameters by answering the questions below.

**Question:** What loss function, metrics and optimizer did you use and why?

**Your answer:** The problem of machine translation is similar to the problem of multi-class classification, where the number of words in the vocabulary is the number of categories that we have. So at each timestep, we want to predict the same class as the ground truth. For this reason, we want to use categorical cross entropy as the loss function that we want to optimize. We set the hidden_dim to 5, because each input sentence contains maximum 5 words. According to the LSTM equations, the number of timesteps is equal to the number of inputs, which is why it is reasonable to set it to 5. As for the embedding dimension, we looked online and found a rule of thumb from [Quora](https://www.quora.com/How-do-I-determine-the-number-of-dimensions-for-word-embedding) that the typical word embedding size is around 128. So we just use that number for training. Adam is the most stable and fast optimizer that we have learned so far.

**Question:** Did you use any Keras callbacks? If so, how did they help you?

**Your answer:** We used Early Stopping and Model Checkpoint. Early stopping is to stop the training when the validation accuracy drops, thus preventing overfitting. And the model checkpoint is to store the best one if we need to stop the training due to time restriction.

**Question:** How did you evaluate that the model was good enough?

**Your answer:** Well, we use 20% of the entire dataset for validation.

#### 2.2.4 Testing <a class="anchor" id="2.2.4"></a>
Test your neural machine translator on a few sentences by running the test case below. The similarity metric is calculated comparing word embeddings.  

You can also use the function `translate` to try your own sentences, just remember that every word needs to be in the vocabulary!

In [14]:
def translate(sentence):
    """ Translate a sentence using `model`. `model` is assumed to be a global variable.
    
    Arguments:
    sentence - a string to translate
    
    Returns:
    the translated sentence as a string
    """
    
    # Check that each of the words in the input sentence exist in the input dictionary
    try:
        x = [eng_word_to_ix[word] for word in sentence.split(' ')]
    except KeyError as e: 
        print('{0} doesn\'t exist in the vocabulary!'.format(e))
        
    # Pad the input sentence
    x = pad_sequences([x], maxlen=SEQ_MAX_LEN, dtype='int32')
    
    # YOUR CODE HERE
    from keras.models import load_model
    model = load_model('./models/best_model.h5')
    ans = model.predict([x, np.zeros(x.shape[0])])
    ans = ans.reshape(ans.shape[1], ans.shape[2])
    ans = np.argmax(ans, axis=1)
    y_pred_words = ' '.join(spa_ix_to_word[ind] for ind in ans if ind != 0)
    
    return y_pred_words


In [ ]:
# test case. be sure the variable model has a reference to your trained model
test_predictions(translate, eng_word_to_ix, spa_ix_to_word, SEQ_MAX_LEN, model)

In [ ]:
# test for yourself
print(translate('this is a dog .'))

In [ ]:
print(translate('she love apples .'))

In [ ]:
print(translate('he opened the window .'))

In [ ]:
print(translate('this is working well .'))

In [ ]:
print(translate('i am a great student .'))

## Congratulations!
You have successfully implemented a recurrent neural network from scratch using only NumPy and also implemented a neural machine translator using Keras!

When choosing the architecture for your neural machine translator you are partly restricted from the capabilities of Keras and partly restricted from your available computing power.  

**Question:** Give 3 suggestions for different techniques that can be used to improve your neural machine translator.

**Your answer:** Our 3 suggestions are: Stacking LSTMs (with more layers), Adding regularization (to reduce the variance, and thus overfitting), and Using transfer learning.

Stacking LSTMs means that we add more LSTMs layers on top of each other. This gives us the ability to capture non-linearity of machine translation.

The reason why regularization might help is because our results seem a bit overfit. We could use something like L2 regularizations, to make sure that the values of the parameters are small